In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, DateType, FloatType, IntegerType, TimestampType, ArrayType, StructType, StructField
from pyspark.sql.functions import from_unixtime, sum, rank,lag, explode, expr,spark_partition_id, to_date, coalesce, lit, to_timestamp, col, month, concat, count, max, when, dayofweek, datediff,dense_rank, desc, date_format
import pyspark.sql.functions as F
from pyspark.sql.window import Window
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import re
from pyspark.ml.feature import Tokenizer, StopWordsRemover, Word2Vec,HashingTF,IDF, CountVectorizer,VectorAssembler
from pyspark.sql.functions import udf
from pyspark.ml.feature import Tokenizer, CountVectorizer, IDF
from pyspark.ml import Pipeline,PipelineModel
from sparknlp.base import DocumentAssembler, Finisher
from sparknlp.annotator import LemmatizerModel
from pyspark.ml.classification import LinearSVC, LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator,BinaryClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
import sparknlp
import warnings
from google.cloud import storage
import os


In [6]:
spark = SparkSession.builder.appName("AddLabelColumn").getOrCreate()

# Get the context of the Pyspark environment
spark.sparkContext.getConf().getAll()

# Store spark context as a variable
sc = spark.sparkContext

stoarge_client = storage.Client()


In [7]:
def list_csv_files(bucket_name, folder_name):
    bucket = stoarge_client.bucket(bucket_name)
    # list folder with csv suffix and make sure the subdirectory file is not listed (so merged file will not be concated in the merged_df)
    return [("gs://"+ bucket_name + "/" +blob.name) for blob in bucket.list_blobs(prefix=folder_name) if blob.name.endswith('.csv') and not '/' in blob.name[len(folder_name):]]

project_name = "msca-bdp-student-ap"
bucket_name = 'msca-bdp-student-gcs'
folder_name = 'Group2_Final_Project/scrapy_data_youtube/'

dir_file_lst = list_csv_files(bucket_name, folder_name)
print(dir_file_lst)

['gs://msca-bdp-student-gcs/Group2_Final_Project/scrapy_data_youtube/comments_Books.csv', 'gs://msca-bdp-student-gcs/Group2_Final_Project/scrapy_data_youtube/comments_Financial Advice.csv', 'gs://msca-bdp-student-gcs/Group2_Final_Project/scrapy_data_youtube/comments_Relationship.csv', 'gs://msca-bdp-student-gcs/Group2_Final_Project/scrapy_data_youtube/comments_interesting.csv', 'gs://msca-bdp-student-gcs/Group2_Final_Project/scrapy_data_youtube/comments_life tips.csv']


In [8]:
def merge_youtube_df(dir_file_lst, labelCol, saved_path):
    res_df = None

    for file_path in dir_file_lst:
        splitted_path = file_path.split("/")
        file_name = splitted_path[-1].split(".")[0]
        file_name = file_name.split("_")[1]

        curr_df = spark.read \
                    .option("quote", "\"") \
                    .option("escape", "\"") \
                    .option("multiLine", True) \
                    .option("ignoreLeadingWhiteSpace", True) \
                    .csv(file_path, header=True, inferSchema=True)
        curr_df = curr_df.withColumn(labelCol, lit(file_name))

        if res_df is None:
            res_df = curr_df
        else:
            res_df = res_df.union(curr_df)
    
    res_pd = res_df.toPandas()
    res_pd.to_csv(saved_path,index=False)
            
#    res_df.write.mode("overwrite").options(delimiter=";", quote='"', escape='"').csv(saved_path, header=True)
     
   
    return res_df

# Usage
labelCol = "tag"
saved_path = "gs://msca-bdp-student-gcs/Group2_Final_Project/scrapy_data_youtube/merged/youtube_comment.csv"
merged_df = merge_youtube_df(dir_file_lst, labelCol,saved_path)
merged_df.select(col(labelCol)).show()


+-----+
|  tag|
+-----+
|Books|
|Books|
|Books|
|Books|
|Books|
|Books|
|Books|
|Books|
|Books|
|Books|
|Books|
|Books|
|Books|
|Books|
|Books|
|Books|
|Books|
|Books|
|Books|
|Books|
+-----+
only showing top 20 rows



In [9]:
youtube_comments = spark.read \
            .option("quote", "\"") \
            .option("escape", "\"") \
            .option("multiLine", True) \
            .option("ignoreLeadingWhiteSpace", True) \
            .csv('gs://msca-bdp-student-gcs/Group2_Final_Project/scrapy_data_youtube/merged/youtube_comment.csv', header=True, inferSchema=True)

youtube_comments.printSchema()
youtube_comments_grouped = youtube_comments.select(col("Comments"),col("Likes"),col("tag")).groupBy(col("tag")).agg(count("*").alias("tag_count"))
youtube_comments_grouped.show()

youtube_comments_modified = youtube_comments.withColumn(
    "subreddit_name_prefixed",
    when(col("tag") == "problem-solving", "r/YouShouldKnow")
    .when(col("tag") == "Relationship", "r/relationship_advice")
    .when(col("tag") == "Programming", "r/programming")
    .when(col("tag") == "Financial Advice", "r/personalfinance")
    .when(col("tag") == "Books", "r/Books")
    .when(col("tag") == "interesting", "r/mildlyinteresting")
    .when(col("tag") == "life tips", "r/LifeProTips")
    .otherwise(lit(None))  # Set to null if none of the conditions are met
) \
    .withColumn("body", col("Comments"))

root
 |-- Video Title: string (nullable = true)
 |-- Video URL: string (nullable = true)
 |-- Views: string (nullable = true)
 |-- Publication Date: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Likes: string (nullable = true)
 |-- Channel URL: string (nullable = true)
 |-- Channel Name: string (nullable = true)
 |-- Channel Image: string (nullable = true)
 |-- Channel Subscribers: string (nullable = true)
 |-- Comments: string (nullable = true)
 |-- tag: string (nullable = true)



+----------------+---------+
|             tag|tag_count|
+----------------+---------+
|    Relationship|   240780|
|Financial Advice|    70880|
|           Books|   132824|
|     interesting|   548336|
|       life tips|   196188|
+----------------+---------+



In [10]:
youtube_comments_selected = youtube_comments_modified.select(col("body"),col("Likes"),col("subreddit_name_prefixed"))
res_saved_path = "gs://msca-bdp-student-gcs/Group2_Final_Project/scrapy_data_youtube/merged/youtube_comment_selected.csv"

youtube_comments_selected.toPandas().to_csv(res_saved_path,index=False)

In [11]:
def list_all_files(bucket_name, folder_name):
    bucket = stoarge_client.bucket(bucket_name)
    file_lst = [('r/'+ blob.name.split("/")[2].split("_")[0]) for blob in bucket.list_blobs(prefix = folder_name)]
    file_lst = list(set(file_lst))
  
    return file_lst

In [12]:
bucket_name = 'msca-bdp-student-gcs'
folder_name = 'Group2_Final_Project/modelr/'
trained_tags = list_all_files(bucket_name, folder_name)


print(trained_tags)

['r/askscience', 'r/ifyoulikeblank', 'r/mildlyinteresting', 'r/programming', 'r/lifehacks', 'r/Showerthoughts', 'r/Damnthatsinteresting', 'r/SkincareAddiction', 'r/Documentaries', 'r/malefashionadvice', 'r/IWantToLearn', 'r/Foodforthought', 'r/bestof', 'r/socialskills', 'r/travel', 'r/YouShouldKnow', 'r/changemyview', 'r/suggestmeabook', 'r/tifu', 'r/Games', 'r/AskHistorians', 'r/DIY', 'r/scifi', 'r/gadgets', 'r/EatCheapAndHealthy', 'r/IAmA', 'r/space', 'r/personalfinance', 'r/relationship', 'r/science', 'r/', 'r/UpliftingNews', 'r/books', 'r/philosophy', 'r/Fantasy', 'r/Fitness', 'r/sports', 'r/GetMotivated', 'r/WritingPrompts', 'r/explainlikeimfive', 'r/femalefashionadvice', 'r/gaming', 'r/bodyweightfitness', 'r/podcasts', 'r/LifeProTips', 'r/technology', 'r/buildapc', 'r/history', 'r/gardening', 'r/todayilearned', 'r/boardgames']


In [44]:
# DO NOT DELETE - Pandas approach download and re-upload
# storage_client = storage.Client()

# Process each CSV file and merge them.
# def process_and_merge_csv_files(bucket_name, folder_name):
    
    
#     csv_files = list_csv_files(bucket_name, folder_name)
#     df_lst = []

#     for file_name in csv_files:
#         # Download the file
#         blob = storage_client.bucket(bucket_name).blob(file_name)
#         blob.download_to_filename(os.path.basename(file_name))

#         # Process the file
#         df = pd.read_csv(os.path.basename(file_name))
#         label = os.path.basename(file_name).split('_')[1] if file_name.lower().startswith('comments') else os.path.basename(file_name)
#         df['tag'] = label

#         df_lst.append(df)

#         # Clean up local file
#         os.remove(os.path.basename(file_name))

#     # Merge all dataframes
#     merged_df = pd.concat(df_lst, ignore_index=True)
#     return merged_df

# # Replace with your bucket name and folder name
# bucket_name = 'msca-bdp-student-gcs'
# folder_name = 'Group2_Final_Project/scrapy_data_youtube/'

# # Process and merge CSV files
# merged_df = process_and_merge_csv_files(bucket_name, folder_name)
# merged_df.to_csv('youtube_comments.csv', index=False)

# Save the merged DataFrame as a new CSV file

# blob = storage_client.bucket(bucket_name).blob(folder_name + 'merged/' + 'youtube_comments.csv')
# blob.upload_from_filename('youtube_comments.csv')
